# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
cities_df = pd.read_csv('/Users/amber/Desktop/GitHub/python-api-challenge/output_data/cities.csv')
cities_df.dropna()
cities_df.drop(columns=['Unnamed: 0'], inplace=True)
cities_df

,City,Country,High Temp (F),Cloudiness (%),Humidity (%),Wind (mph),Latitude,Longitude
0,Bubaque,GW,78.64,3,68,21.16,11.2833,-15.8333
1,Ostrovnoy,RU,36.54,100,99,1.66,68.0531,39.5131
2,Ushuaia,AR,55.40,40,58,16.11,-54.8000,-68.3000
3,Castro,BR,74.52,18,48,5.19,-24.7911,-50.0119
4,Kapaa,US,73.40,75,73,17.27,22.0752,-159.3190
...,...,...,...,...,...,...,...,...
525,Salinas,US,55.40,90,87,5.75,36.6777,-121.6555
526,Aracati,BR,88.38,52,55,11.99,-4.5617,-37.7697
527,Bamako,ML,105.80,40,17,9.22,12.6500,-8.0000
528,Bhag,PK,89.73,0,15,3.11,29.0415,67.8239


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# set lat and long as locations
locations = cities_df[["Latitude", "Longitude"]]

# set humidity as weight
humidity = cities_df["Humidity (%)"].astype(float)

# plot the humidity heatmap
fig = gmaps.figure()

# create the heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

# add the head layer
fig.add_layer(heat_layer)

# display the heat map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
# narrow down cities to fit weather conditions
my_vacays = cities_df.loc[(cities_df['High Temp (F)'] <= 85) & (cities_df['Humidity (%)'] <= 40) & (cities_df['Wind (mph)'] <= 10)]
my_vacays.head()

,City,Country,High Temp (F),Cloudiness (%),Humidity (%),Wind (mph),Latitude,Longitude
10,Albany,US,68.00,62,19,8.88,42.6001,-73.9662
13,Bethel,US,68.00,1,26,8.05,41.3712,-73.4140
22,Tura,IN,78.15,65,29,4.32,25.5198,90.2201
43,Winnipeg,CA,42.01,17,24,5.75,49.8844,-97.1470
46,Jamestown,US,61.00,1,33,3.44,42.0970,-79.2353


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [4]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
